# Question 3

## Question 3.1.1

The program is in the next block.

## Question 3.1.2

`hash_code1()` will generate a hash code with the ascii value of the characters with the power of the last digit.

For example, `hash_code1('AAAAAA 1') = 65^1 + 65^1 + 65^1 + 65^1 + 65^1 + 65^1 = 390`

`hash_code2()` does another way, it first multiplies the ascii value of the 6 characters, then the product is divided by the last digit plus 1 (in case of 0). According to the design, this method is supposed to lower the collision compared to `hash_code1()`.

`hash_code1()` and `hash_code2()` take the string 'AAAAAA 1' as input, but both of the two methods divide the input key string into two parts, the six characters and the last digit.

`hash_code3()` utilizes the conventional SHA-1 algorithm in the `hashlib` module. SHA-1 can generate a 40-digit hash code (hex value) regardless of input string length, but in this case, I changed the output hash code into decimal value. 

`hash_code3()` takes the key 'AAAAAA 1' as input and generate the SHA-1 hash code without cutting the string.

`compression()` reads the input hashcode, returns the value of (hashcode) mod 10^6. After the comperssion we can have a million different key values at most.

## Question 3.2.1

The data structure is in the third block.

The method `insert()` and `search()` in class `HashTable` is inspired by week 7 workshop solution.

## Question 3.2.2

collision rate = (collision count / total #records) * 100%

* collision only count once for each record input

The test results:

100 names: block 4

200 names: block 5

500 names: block 6

all names: block 7

## Question 3.2.3

It fits my expectation that `hash_code2()` performs better than `hash_code1()` in the first 3 cases, but in the last scenario (all names), the improvement is subtle and I do not consider it as improvement. These algorithms are simple and can only produce a limited range of hash codes. 

Generally, `hash_code3()` has the best performance. It generates a wide variety of key values, which significantly reduces collision.

In [9]:
# Q 3.1.1

# The body of the hash functions are from W7 workshop instructors' solution

import hashlib

def hash_code1(key):
    hashcode = 1
    if not isinstance(key, str):
        raise TypeError
    
    separatedKeys = key.split(' ')
    # sums up the ascii value of the characters
    for i in range(0, 6):
        hashcode += int(ord(separatedKeys[0][i])) ** (int(separatedKeys[1][0]))

    return hashcode


def hash_code2(key):
    hashcode = 1
    if not isinstance(key, str):
        raise TypeError
    
    separatedKeys = key.split(' ')
    # multiply the ascii value of the characters
    # This should create a huge space
    for i in range(0, 6):
        hashcode *= int(ord(separatedKeys[0][i]))
    # divide by (last digit + 1) and take the quotient only
    hashcode //= int(separatedKeys[1][0]) + 1 # in case of 0
    
    return hashcode

# This function utilizes hashlib, will retuen a decimal sha-1 value
def hash_code3(key):
    if not isinstance(key, str):
        raise TypeError

    s = hashlib.sha1()
    # unicode object has to be encoded first
    s.update(key.encode()) 
    # turn hex into dec
    hashcode = int(s.hexdigest(), 16)
    return hashcode

# hashcode mod 10^6
def compression(hashcode):
    return hashcode % 1000000


In [10]:
# Q 3.2.1
class HashTable:
    def __init__(self, mode):
        # bad strategy, but in this case
        # such approach can ensure enough space 
        # for a large amount of records
        self.SIZE = 1000000
        self.hash_table = [None] * self.SIZE
        # self.mode determines which hash function to run
        self.mode = mode
        self.collision = 0
    
    def insert(self, key, value):
        index = None
        if self.mode == 1:
            index = compression(hash_code1(key))
        elif self.mode == 2:
            index = compression(hash_code2(key))
        elif self.mode == 3:
            index = compression(hash_code3(key))
        else:
            raise ValueError
        
        if self.hash_table[index] is None:
            self.hash_table[index] = (key, value)
        else:
            # linear probing
            while self.hash_table[index] is not None:
                index = (index+1) % self.SIZE
            self.hash_table[index] = (key, value)
            self.collision += 1
        
    def search(self, key):
        index = None
        if self.mode == 1:
            index = compression(hash_code1(key))
        elif self.mode == 2:
            index = compression(hash_code2(key))
        elif self.mode == 3:
            index = compression(hash_code3(key))
        else:
            raise ValueError
        hash_table_index_val = self.hash_table[index]

        if (type(hash_table_index_val) is list):
          for k, val in self.hash_table[index]:
            if k == key:
              return val
        elif (type(hash_table_index_val) is tuple):
          if key == self.hash_table[index][0]:
            return self.hash_table[index][1]

        raise Exception("Entry not Found")

    # habit from java, prevent from modifying the value of self.collision
    def getCollision(self):
        return self.collision


In [11]:
# Q 3.2.2
# Run the two blocks above first.
ht1 = HashTable(1)
ht2 = HashTable(2)
ht3 = HashTable(3)

# 100
f = open("student_records.txt", 'r')

for i in range(100):
    line = f.readline()
    line = line.split(' ')
    key = line[0] + " " + line[1]
    val = line[2][:-1]
    
    ht1.insert(key, val)
    ht2.insert(key, val)
    ht3.insert(key, val)

f.close()

testSize = 100

print('case 1: 100 names')
print('function 1:', (ht1.getCollision() / testSize) * 100, '%')
print('function 2:', (ht2.getCollision() / testSize) * 100, '%')
print('function 3:', (ht3.getCollision() / testSize) * 100, '%')


case 1: 100 names
function 1: 5.0 %
function 2: 0.0 %
function 3: 0.0 %


In [12]:
# 200
ht1 = HashTable(1)
ht2 = HashTable(2)
ht3 = HashTable(3)

f = open("student_records.txt", 'r')

for i in range(200):
    line = f.readline()
    line = line.split(' ')
    key = line[0] + " " + line[1]
    val = line[2][:-1]
    
    ht1.insert(key, val)
    ht2.insert(key, val)
    ht3.insert(key, val)

f.close()

testSize = 200

print('case 2: 200 names')
print('function 1:', (ht1.getCollision() / testSize) * 100, '%')
print('function 2:', (ht2.getCollision() / testSize) * 100, '%')
print('function 3:', (ht3.getCollision() / testSize) * 100, '%')


case 2: 200 names
function 1: 8.5 %
function 2: 0.0 %
function 3: 0.0 %


In [13]:
# 500
ht1 = HashTable(1)
ht2 = HashTable(2)
ht3 = HashTable(3)

f = open("student_records.txt", 'r')

for i in range(500):
    line = f.readline()
    line = line.split(' ')
    key = line[0] + " " + line[1]
    val = line[2][:-1]
    
    ht1.insert(key, val)
    ht2.insert(key, val)
    ht3.insert(key, val)

f.close()

testSize = 500

print('case 3: 500 names')
print('function 1:', (ht1.getCollision() / testSize) * 100, '%')
print('function 2:', (ht2.getCollision() / testSize) * 100, '%')
print('function 3:', (ht3.getCollision() / testSize) * 100, '%')

print(ht1.search('ZTDVPT 1'))
print(ht2.search('ZTDVPT 1'))
print(ht3.search('ZTDVPT 1'))


case 3: 500 names
function 1: 15.0 %
function 2: 1.6 %
function 3: 0.0 %
Michael
Michael
Michael


In [14]:
# all
ht1 = HashTable(1)
ht2 = HashTable(2)
ht3 = HashTable(3)

f = open("student_records.txt", 'r')
lines = f.readlines()
f.close()

testSize = 0

for i in lines:
    i = i.split(' ')
    key = i[0] + " " + i[1]
    val = i[2][:-1]
    
    ht1.insert(key, val)
    ht2.insert(key, val)
    ht3.insert(key, val)
    testSize += 1

print('case 4: all names')
print('function 1:', (ht1.getCollision() / testSize) * 100, '%')
print('function 2:', (ht2.getCollision() / testSize) * 100, '%')
print('function 3:', (ht3.getCollision() / testSize) * 100, '%')


case 4: all names
function 1: 24.61757771807665 %
function 2: 21.042820330061957 %
function 3: 1.1075168594769451 %


# Question 4

## Definition:

* Random order: the input array is not sorted and is randomly generated.

* Nearly sorted: the input array is almost sorted except for a few elements that are misplaced.

* reversed order: the input array is sorted, but its order is opposite to the expected one.

## Question 4.1.1

With the random ordered input, quick sort provides the average time complexity of `O(n logn)`. This is the sum of the time usage considering all possible pivot then divide by `n`. Although quick sort has the worst-case time complexity of `O(n^2)`, we do not really concern it since it is unlikely to happen in real world.

## Question 4.1.2

Insertion sort compares the selected element to the ones of the sorted sequence from its tail.

The average time complexity of insertion sort is `O(n^2)`. Yet insertion sort takes linear time when the input array is nearly sorted. In such situation it is one of the best compare-based sorting algorithms to use.

The other choices have bigger time complesity than the one of insertion sort.

## Question 4.1.3

Usually we take the reversed input as the worst case. Since it is inevitable for discussing worst-case performance when it comes to algorithm, a reversed ordered input array is also a popular issue.

Heap sort offers the worst-case time complexity of `O(n logn)`, which outperfroms the other kinds of sorting algorithms in this question.

## Question 4.2.4

The program is in the next block.

## Question 4.2.5

a. 

First of all, the merge sort algorithm breaks down the input array recursively until every segment only contains one element. Then compare the two segments in each group to determine the internal order and merge the two elements into one. After that we can get n/2 groups of arrays which have length of 2, then we repeat the step until the sorted sequences are merged back to a single array.

b. 

Both merge sort and quick sort belong to divide and conquor algorithm. The difference between them is how to divide the array into small pieces. Quick sort chooses pivots randomly in every recursion then sorts the subsequences, while merge sort mostly recursively chooses the middle point of the array as the pivot and divide it, and then swap the elements.

Though merge sort has a more acceptable worst-case time complexity `O(n logn)` than the one of quick sort `O(n^2)`, we cannot take the worst case too seriously since the worst case for quick sort rarely happens. In fact, they share the similar average time complexity of `O(n logn)`

Merge sort is a stable sorting algorithm, while the quick sort is not. If we want to stablize a non-stable algorithm, it usually cost some extra space.


In [15]:
# Q 4.2.4

def merge(left, right):
    if not len(left) or not len(right):
        return left or right
    result = []
    i, j = 0, 0
    while len(result) < len(left) + len(right):
        if left[i] > right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
        if i == len(left) or j == len(right):
            result.extend(left[i:] or right[j:])
            break
    return result

def merge_sort(arr):
    if len(arr) < 2:
        return arr
    middle = int(len(arr) / 2)
    left = merge_sort(arr[:middle])
    right = merge_sort(arr[middle:])
    return merge(left, right)

merge_sort([2, 8, 10, 1, 5])

[10, 8, 5, 2, 1]

# Question 5

## Question 5.1

The program is in the next block.

## Question 5.2.1

I chose the Dijkstra's algorithm to solve this problem.

Dijkstra's algorithm is a greedy search, which always keep the best result for a single step, and expects the overall result to be the optimized one. For non-negative weighted edges in a graph, Dijkstra's algorithm can return a shortest path from one node to another.

At first I thought a simpler search algorithm like BFS or DFS can deal with it easily. But I found it hard to design it to obtain the shortest path. They can reach the end, but the route is not always the shortest one. I thought the situatioin might be realtive to how the graph is generated.

## Question 5.2.2

Dijkstra's algorithm can cope with a graph with non-negative weighted edges.

Therefore the program can also deal with this situation as long as the weight is not negative.

In Graph.dijkstra():

    for v in range(self.V):
            if self.graph[u][v] > 0 and sptSet[v] == False and dist[v] > dist[u] + self.graph[u][v]:
                dist[v] = dist[u] + self.graph[u][v]

The weights are still counted, so the only job left to make the weights to take effect is to update `give_me_the_shortest_path()`, make it generate a graph with real weights instead of unified weights.


In [16]:
# Q 5.1
import sys

class Graph():
 
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]
        
    def minDistance(self, dist, sptSet):
 
        # Initialize minimum distance for next node
        # It serves as inf
        min = sys.maxsize

        # Looks for the neatest vertex that is not in the
        # shortest path list
        for v in range(self.V):
            if dist[v] < min and sptSet[v] == False:
                min = dist[v]
                min_index = v
 
        return min_index
 
    # Dijkstra's algorithm implementation
    def dijkstra(self, src):
 
        dist = [sys.maxsize] * self.V
        dist[src] = 0
        sptSet = [False] * self.V
 
        for i in range(self.V):
            
            # Pick the shortest distance vertex from the map that is not processed yet.
            # u is always equal to src in first iteration
            u = self.minDistance(dist, sptSet)
 
            # Put the minimum distance vertex in the
            # shortest path tree
            sptSet[u] = True
 
            # Update dist value of the nearby vertices
            # of the picked vertex 
            # if currnet dist is greater than the new one
            # and the vertex is not pushed into the shortest
            # path list.
            for v in range(self.V):
                if self.graph[u][v] > 0 and sptSet[v] == False and dist[v] > dist[u] + self.graph[u][v]:
                    dist[v] = dist[u] + self.graph[u][v]

        return dist[self.V - 1]


def give_me_the_shortest_path(n):
    # generate graph according to N
    if n < 2 or n > 50:
        raise ValueError

    roadMap = Graph(n)

    for i in range(n):
        if i + 1 >= n:
            roadMap.graph[i][i] = 0
        elif (i + 1) * 3 > n:
            roadMap.graph[i][i + 1] = 1
        else:
            roadMap.graph[i][i + 1] = 1
            roadMap.graph[i][(i+1)*3-1] = 1

    return roadMap.dijkstra(0)

give_me_the_shortest_path(50)

8